#### 제1유형

In [21]:
# 문제 1 - 1
import pandas as pd

df = pd.read_csv("data6-1-1.csv")
# df

# df.info()
# df.head()
# df['소방서'].value_counts()

df['도착시간'] = pd.to_datetime(df['도착시간'])
df['출동시간'] = pd.to_datetime(df['출동시간'])

# df.info()
df['시간차이(분)'] = (df['도착시간'] - df['출동시간']).dt.total_seconds() / 60
df

avg_diff = df.groupby('소방서')['시간차이(분)'].mean()
print(avg_diff)
idx = avg_diff.idxmax()
print(idx)
result = avg_diff[idx]
print(round(result))

소방서
소방서1     54.952381
소방서10    42.365000
소방서2     60.228571
소방서3     43.490741
소방서4     49.136667
소방서5     61.250000
소방서6     41.764583
소방서7     56.670370
소방서8     42.693750
소방서9     80.658889
Name: 시간차이(분), dtype: float64
소방서9
81


In [40]:
  # 문제1 - 2
import pandas as pd

df = pd.read_csv("data6-1-2.csv")
df

df['총학생수'] = df.iloc[:, 2:].sum(axis=1)
df

# 교사 한 명당 학생수
df['교사당 학생수'] = df['총학생수'] / df['교사수']
print(df)

idx = df['교사당 학생수'].idxmax()
print(idx)

# 해당 학교의 전체 교사 수
print(df.loc[idx, '교사수'])

      학교명  교사수  1학년  2학년  3학년  4학년  5학년  6학년  총학생수    교사당 학생수
0     학교1   20  118   72  154  118   77   59   598  29.900000
1     학교2   31  187  193  135  137   63   90   805  25.967742
2     학교3   23  136  193  135  112   91   50   717  31.173913
3     학교4   32   72  146   67   51  130  164   630  19.687500
4     학교5   22   61   73   86   82   55  124   481  21.863636
..    ...  ...  ...  ...  ...  ...  ...  ...   ...        ...
95   학교96   38   75  166  113  191  172   57   774  20.368421
96   학교97   30  125  169  145  122  187  194   942  31.400000
97   학교98   37  136  100   93   60  145   63   597  16.135135
98   학교99   37   74  175  145  157   95   76   722  19.513514
99  학교100   40   68  109  196  148   84  179   784  19.600000

[100 rows x 10 columns]
7
19


In [54]:
# 문제1 - 3
import pandas as pd

df = pd.read_csv("data6-1-3.csv")
df

df['총 범죄 건수'] = df.iloc[:, 1:7].sum(axis=1)
df

# 연도별 총 범죄 건수의 합
df['연도'] = df['날짜'].str[:4]
df

result = df['총 범죄 건수'].groupby(df['연도']).sum()
print(result)

print(round(result.max() / 12))

연도
2020    6288
2021    6318
2022    6179
2023    6394
2024    6200
Name: 총 범죄 건수, dtype: int64
533


#### 제2유형

In [79]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

train = pd.read_csv("energy_train.csv")
test = pd.read_csv("energy_test.csv")

# train.info()
# test.info()

# train['Heat_Load'].value_counts()

train.isnull().sum()
print(train.shape, test.shape)

# 데이터 선택
target = train.pop("Heat_Load")

# 원-핫 인코딩
print(train.shape, test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# train.info()
print(train.shape, test.shape)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(train,
            target, test_size=0.2, random_state=303)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 모델 생성 및 학습
model = RandomForestClassifier(random_state=303)
model.fit(X_train, y_train)
pred = model.predict(X_test)

# 평가
f1 = f1_score(y_test, pred, average='micro')
print("f1-score", f1)

# 테스트 데이터 평가
pred2 = model.predict(test)

# 파일 생성
submit = pd.DataFrame({"pred": pred2})
submit.to_csv("result.csv", index=False)

pd.read_csv("result.csv")

(537, 10) (231, 9)
(537, 9) (231, 9)
(537, 16) (231, 16)
(429, 16) (108, 16) (429,) (108,)
f1-score 0.9444444444444444


pred
0          Low
1         High
2         High
3          Low
4          Low
..         ...
226   Very Low
227     Medium
228   Very Low
229        Low
230  Very High

[231 rows x 1 columns]

#### 3유형

In [24]:
# 1 -1
import pandas as pd

df = pd.DataFrame({
    "항암약": [4, 4, 3, 4, 1, 4, 1, 4, 1, 4, 4, 2, 1, 4, 2, 3, 2, 4, 4, 4]
})
df.head()
print(len(df))


# 귀무가설 : 감기약의 부작용과 항암약의 부작용은 동일하다.
# 대립가설 : 감기약의 부작용과 항암약의 부작용은 다르다.

# 이상 없음의 빈도 계산
cond = df["항암약"] == 4
print(sum(cond))

# 항암약을 투여 받은 환자 중 "이상 없음" 비율 계산
ratio = sum(cond) / len(df)
print(ratio)

# 카테고리별 비율 게산
df["항암약"].value_counts(normalize=True)

# 1 - 2
from scipy.stats import chisquare

# 각 카테고리의 비율을 리스트로 만들기
prob = [0.1, 0.05, 0.15, 0.7]

# 기대 빈도수 계산
# expected_counts = [0.1*20, 0.05*20, 0.15*20, 0.7*20]
# print(expected_counts)

expected_counts = [x * len(df) for x in prob]
print(expected_counts)

# 관찰값 계산
observed_counts = df["항암약"].value_counts().sort_index().to_list()
print(observed_counts)

# 카이제곱 검정 수행
result = chisquare(f_obs=observed_counts, f_exp=expected_counts)
print(result)
print(result.pvalue)

20
11
0.55
[2.0, 1.0, 3.0, 14.0]
[4, 3, 2, 11]
Power_divergenceResult(statistic=6.976190476190476, pvalue=0.07266054733847573)
0.07266054733847573


In [44]:
# 1 - 2
import pandas as pd

df = pd.read_csv("data6-3-2.csv")
df

# 선형회귀 모델 구축
from statsmodels.formula.api import ols

formula = 'temperature ~ solar + wind + o3'
model = ols(formula, df).fit()
print(model.summary())

print(model.params['o3'])

print(model.pvalues['wind'])

# 예측 - 새 데이터를 데이터프레임으로 만들기
new_data = pd.DataFrame({
    'solar': [100],
    'wind': [5],
    'o3': [30]
})
new_data

pred = model.predict(new_data)
print(pred[0])

                            OLS Regression Results                            
Dep. Variable:            temperature   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.464
Date:                Thu, 05 Dec 2024   Prob (F-statistic):              0.229
Time:                        18:34:56   Log-Likelihood:                -195.45
No. Observations:                 100   AIC:                             398.9
Df Residuals:                      96   BIC:                             409.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.0507      1.994      9.555      0.0